In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd

In [ ]:
def leaflet_plot_stations(binsize, file):

    station_locations_by_hash = pd.read_csv(file) # data/C2A2_data/BinSize_d
    print(station_locations_by_hash)
#     station_locations_by_hash = df[df['hash'] == hashid]

    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()

    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)

    return mplleaflet.display()

In [ ]:
!find .. | grep -i fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89

In [ ]:
file = '../_data/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv'

In [ ]:
# leaflet_plot_stations(400, file)

### Import data

In [ ]:
df_GHCN = pd.read_csv(file)
df_GHCN.sample(5)

### Convert long DF to wide DF; TMIN and TMAX to columns

In [ ]:
df = df_GHCN.groupby(['Element', 'ID', 'Date']).agg({'Data_Value':min, 'Data_Value':max})
df.unstack(level=0).reset_index()

#### Create separate dataframes for each Element

In [ ]:
df_GHCN_max = df_GHCN[df_GHCN.loc[:, 'Element'] == 'TMAX']
df_GHCN_min = df_GHCN[df_GHCN.loc[:, 'Element'] == 'TMIN']

#### Merge separate dataframes

In [ ]:
df_GHCN = pd.merge(df_GHCN_min, df_GHCN_max, how='outer', left_on=['ID','Date'], right_on=['ID','Date'])
df_GHCN.columns = ['ID', 'Date', 'x', 'TMIN', 'y', 'TMAX']
df_GHCN.drop(['x', 'y'], axis=1, inplace=True)
df_GHCN.sample(10)

In [ ]:
# Convert dates
# observation_dates = np.arange('2017-01-01', '2017-01-09', dtype='datetime64[D]')
df_GHCN.loc[:, 'Date'] = list(map(pd.to_datetime, df_GHCN.loc[:, 'Date'])) 
df_GHCN.loc[:, 'Day of Year'] = df_GHCN.loc[:, 'Date'].dt.dayofyear
df_GHCN.loc[:, 'Day'] = df_GHCN.loc[:, 'Date'].dt.day
df_GHCN.loc[:, 'Month'] = df_GHCN.loc[:, 'Date'].dt.month
df_GHCN.loc[:, 'Year'] = df_GHCN.loc[:, 'Date'].dt.year
# df_GHCN.dtypes

In [ ]:
# Mask leap day
mask_leap = (df_GHCN.loc[:, 'Month'] == 2) & (df_GHCN.loc[:, 'Day'] == 29)
df_GHCN = df_GHCN[~mask_leap]

In [ ]:
# Split df
df_2005 = df_GHCN[df_GHCN.loc[:, 'Year'] < 2015]
df_2015 = df_GHCN[df_GHCN.loc[:, 'Year'] == 2015]

In [ ]:
df_2015.groupby('Day of Year').agg({'TMIN':min, 'TMAX':max})

In [ ]:
# Daily records
daily_records = df_2005.groupby(['Day of Year']).agg({'TMIN':min, 'TMAX':max})
daily_records;

In [ ]:
df_2015.loc[:, 'TMIN'];

In [ ]:


min_record_2015_mask = df_2015.loc[:, 'TMIN'] < daily_records.loc[:, 'TMIN']
print(min_record_2015_mask)

In [ ]:
# Merge
df_2005 = pd.merge(df_2005, daily_records, how='left', left_on='Day of Year', right_index=True)
df_2005.columns = ['ID', 'Date', 'TMIN', 'TMAX', 'Day of Year', 'Day', 'Month', 'Year', 'Record Min', 'Record Max']
df_2005.head()

In [ ]:










# df_2015_ = pd.merge(df_2015, daily_min, how='left', left_on='Day of Year', right_index=True)
# df_2015_ = pd.merge(df_2015_, daily_max, how='left', left_on='Day of Year', right_index=True)
# df_2015_.columns  = ['ID', 'Date', 'TMIN', 'TMAX', 'Day of Year', 'Day', 'Month', 'Year',
#        'Record Min', 'Record Max']
# df_2015_.head()
# df_2005.loc[:, 'Record Min'] = df_2005.groupby('Day of Year').agg({'TMIN':min})  #['TMIN'].transform('min')  #
# df_2005.loc[:, 'Record Max'] = df_2005.groupby('Day of Year')['TMAX'].transform('max')
# df_2005.head(-1)

# # Is record
# df_2005.loc[:, 'Is Min Record'] = df_2005.loc[:, 'TMIN'] == df_2005.loc[:, 'Record Min'] #df_GHCN.groupby('Day of Year')['TMIN'].transform('min')
# df_2005.loc[:, 'Is Max Record'] = df_2005.loc[:, 'TMAX'] == df_2005.loc[:, 'Record Max'] #df_GHCN.groupby('Day of Year')['TMAX'].transform('max')


# # Sanity check leap filter
# # print(df_GHCN[mask_leap])
# df_2005.head(-1)

In [ ]:
plt.figure(figsize=(20, 6))

In [ ]:
#
df_2005_2014 = df_GHCN[(df_GHCN['Year'] > 2004) & (df_GHCN['Year'] < 2015)]
df_2005_2014.head(-1)

df_2015 = df_GHCN[df_GHCN['Year'] == 2015]
df_2015.head(-1)

#
df_min = df_2005_2014.groupby(['Month', 'Day']).agg({'TMIN':min, 'TMAX':max})
df_min.head(-1)

#
df_2015_min = df_2015.groupby(['Month', 'Day']).agg({'TMIN':min, 'TMAX':max})
# assert df_2015[df_GHCN['TMIN'] < df_GHCN['Day Min']] == df_2015_min             
df_2015_min.head(-1)

#
mask = (df_GHCN['Date'].dt.month == 12) & (df_GHCN['Date'].dt.day == 30)
print(df_GHCN[mask])

# #
# plt.plot(df_min.index, df_min['TMIN'], '-')
# plt.plot(df_min.index, df_min['TMAX'], '-') 
# plt.xticks = '{}-{}'.format(df_min['Day'], df_min['Month'])
# # print(xlabel)

# # fill the area between the linear data and exponential data
# plt.gca().fill_between(range(len(df_min.index)), 
#                        df_min['TMIN'], df_min['TMAX'], 
#                        facecolor='gray', 
#                        alpha=0.25)
# # plt.scatter(df_min.index, df_min['TMIN'], '-')

# plt.show()


In [ ]:
df_2015_min = df_GHCN[(df_GHCN['Year'] == 2015) & (df_GHCN['TMIN'] < df_GHCN['Day Min'])]
df_2015_min.head(-1)

In [ ]:
#
df_2005_2014 = df_GHCN[(df_GHCN['Year'] > 2004) & (df_GHCN['Year'] < 2015)]
df_2005_2014.head(-1)

df_2015 = df_GHCN[df_GHCN['Year'] == 2015]
df_2015.head(-1)

#
df_min = df_2005_2014.groupby(['Month', 'Day']).agg({'TMIN':min, 'TMAX':max})
df_min.head(-1)

#
df_2015_min = df_2015.groupby(['Month', 'Day']).agg({'TMIN':min, 'TMAX':max})
# assert df_2015[df_GHCN['TMIN'] < df_GHCN['Day Min']] == df_2015_min             
df_2015_min.head(-1)

#
mask = (df_GHCN['Date'].dt.month == 12) & (df_GHCN['Date'].dt.day == 30)
print(df_GHCN[mask])

# #
# plt.plot(df_min.index, df_min['TMIN'], '-')
# plt.plot(df_min.index, df_min['TMAX'], '-') 
# plt.xticks = '{}-{}'.format(df_min['Day'], df_min['Month'])
# # print(xlabel)

# # fill the area between the linear data and exponential data
# plt.gca().fill_between(range(len(df_min.index)), 
#                        df_min['TMIN'], df_min['TMAX'], 
#                        facecolor='gray', 
#                        alpha=0.25)
# # plt.scatter(df_min.index, df_min['TMIN'], '-')

# plt.show()


In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd

plt.figure()
plt.xlabel("X")
plt.ylabel("Y")
plt.boxplot([1, 1, 2, 3, 4])
plt.xticks([1,2,3,4], ["a", "b", "c", "d"])
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import mplleaflet
import calendar
import datetime
import numpy as np
import pandas as pd

# restore plot defaults
plt.rcdefaults()

# Import data
df_GHCN = pd.read_csv('fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv')

# Convert long DF to wide DF; TMIN and TMAX to columns
df_GHCN_max = df_GHCN[df_GHCN.loc[:, 'Element'] == 'TMAX']
df_GHCN_min = df_GHCN[df_GHCN.loc[:, 'Element'] == 'TMIN']
# df_GHCN_max.head(50)

df_GHCN = pd.merge(df_GHCN_min, df_GHCN_max, how='outer', left_on=['ID','Date'], right_on=['ID','Date'])
df_GHCN.columns = ['ID', 'Date', 'x', 'TMIN', 'y', 'TMAX']
df_GHCN.drop(['x', 'y'], axis=1, inplace=True)
# df_GHCN.head(10)

# Convert dates
# observation_dates = np.arange('2017-01-01', '2017-01-09', dtype='datetime64[D]')
df_GHCN.loc[:, 'Date'] = list(map(pd.to_datetime, df_GHCN.loc[:, 'Date']))
df_GHCN.loc[:, 'Day Month'] = df_GHCN.loc[:, 'Date'].dt.strftime('%d/%b')
df_GHCN.loc[:, 'Day of Year'] = df_GHCN.loc[:, 'Date'].dt.dayofyear
df_GHCN.loc[:, 'Day'] = df_GHCN.loc[:, 'Date'].dt.day
df_GHCN.loc[:, 'Month'] = df_GHCN.loc[:, 'Date'].dt.month
df_GHCN.loc[:, 'Year'] = df_GHCN.loc[:, 'Date'].dt.year
# df_GHCN.dtypes

#
df_GHCN.sort_values(['Date', 'ID'], inplace=True)

# Mask leap day
mask_leap = (df_GHCN.loc[:, 'Month'] == 2) & (df_GHCN.loc[:, 'Day'] == 29)
df_GHCN = df_GHCN[~mask_leap]

# Split df
df_2005 = df_GHCN[df_GHCN.loc[:, 'Year'] < 2015]
df_2015 = df_GHCN[df_GHCN.loc[:, 'Year'] == 2015]

# Daily records
daily_records_2005 = df_2005.groupby(['Day Month']).agg({'TMIN':min, 'TMAX':max})
daily_records_2015 = df_2015.groupby(['Day Month']).agg({'TMIN':min, 'TMAX':max})
lower = daily_records_2015.loc[:, 'TMIN'] < daily_records_2005.loc[:, 'TMIN']
higher = daily_records_2015.loc[:, 'TMAX'] > daily_records_2005.loc[:, 'TMAX']

# Merge
df_2015 = pd.merge(df_2015, daily_records_2005, how='left', left_on=['Day Month'], right_index=True)
df_2015.columns = ['ID', 'Date', 'TMIN', 'TMAX', 'Day Month', 'Day of Year', 'Day', 'Month', 'Year', 'Low', 'High']
df_2015.head(-1)

#
plt.figure(figsize=(20, 6))
# fig, ax = plt.subplots()
ax = plt.gca()
fig = plt.gcf()
## Rotate date labels automatically
fig.autofmt_xdate()

df_plot = df_2015.groupby('Day of Year').agg({'TMIN':min, 'TMAX':max, 'Low':min, 'High':max})
df_scatter_min = df_plot[df_plot.loc[:, 'TMIN'] < df_plot.loc[:, 'Low']]
df_scatter_max = df_plot[df_plot.loc[:, 'TMAX'] > df_plot.loc[:, 'High']]

plt.plot(df_plot.index.values, df_plot.loc[:, 'Low'], '-')
plt.plot(df_plot.index.values, df_plot.loc[:, 'High'], '-')

#
plt.xlabel('Day of year')
plt.ylabel('Temperature in tens grades of Celcius')
plt.title('Daily Temperature Extremes \n2015 vs 2005-2014')

#
# fill the area between the linear data and exponential data
plt.gca().fill_between(range(len(df_plot.index)),
                       df_plot.loc[:, 'Low'], df_plot.loc[:, 'High'],
                       facecolor='gray',
                       alpha=0.25)
plt.scatter(df_scatter_min.index, df_scatter_min.loc[:, 'TMIN'], c='red')
plt.scatter(df_scatter_max.index, df_scatter_max.loc[:, 'TMAX'], c='red')

#

myFmt = DateFormatter("%d/%b")
ax.xaxis_date()
ax.xaxis.set_major_formatter(myFmt)
ax.xaxis.set_major_locator(mdates.MonthLocator())
fig.autofmt_xdate()
plt.xticks(np.arange(12), rotation=17)
# plt.xticks([])
# plt.yticks([])
plt.setp(ax.get_xticklabels(), visible=True)
plt.show()

In [ ]:
import matplotlib.dates as mdates

N = 20
numpy.random.seed(N)

df = pandas.DataFrame(
    data=numpy.random.randn(N), 
    index=pandas.date_range('1/1/2014', periods=N, freq='m'),
    columns=['A']
)

fig, (ax1, ax2) = plt.subplots(1,2, figsize=(15, 10))

ax1.bar(df.index, df['A'], width=25, align='center')
ax2.barh(df.index, df['A'], height=25, align='center')

# Spines
for ax in [ax1, ax2]:
    for pos in 'left top right bottom'.split():
        ax.spines[pos].set_visible(False)

# 
ax1.set_yticks([])
ax2.set_xticks([])
    
# Text
for i, date, A in zip(range(df.shape[0]), df.index, df.A):
    ax1.text(mdates.date2num(date)-6, A+0.2, round(A*10,0), color='k', size=10, fontweight='bold', rotation=90)
    ax1.tick_params(axis='x', labelrotation=90)


# Text
for i, date, A in zip(range(df.shape[0]), df.index, df.A):
    ax2.text(A+0.05, mdates.date2num(date)-4, round(A*10,0), color='k', size=10, fontweight='bold')
    

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b-%Y'));
ax1.spines.clear

# ax1.set_xticklabels(mdates.date2num(df.index), rotation=45)
ax2.yaxis.set_major_formatter(mdates.DateFormatter('%b-%Y'));


# fig.autofmt_xdate();

#     ax1.tick_params(axis='y',          # changes apply to the y-axis
#                     which='both',      # both major and minor ticks are affected
#                     bottom=False,      # ticks along the bottom edge are off
#                     top=False,         # ticks along the top edge are off
#                     labelbottom=False) # labels along the bottom edge are off

In [ ]:
# I found a workaround, by applying this at the end:
# In this case, df is the DataFrame I'm plotting, and d.index is the DatetimeIndex.
plt.gca().xaxis.set_major_formatter(plt.FixedFormatter(df.index.to_series().dt.strftime("%b %Y")))

In [ ]:
# Just FYI, if you have stored the axis for the plot already (as in #1918 (comment)), you can also do
ax.xaxis.set_major_formatter(plt.FixedFormatter(df.index.to_series().dt.strftime("%d %b %Y")))

In [ ]:
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off